In [1]:
!pip install pyreadr

In [2]:
import pyreadr

## Helper functions

In [3]:
def convert_nonzero_to_one(feature):
    df[feature] = df[feature].apply(lambda x: 1 if x != 0 else x)
    return df

In [4]:
def drop_features(drop_features):
    df_final = df.drop(drop_features, axis=1)
    return df_final

## Upload dataset

In [5]:
dataset_name = "credit"

# Load the RDA file
result = pyreadr.read_r(dataset_name +".rda")

# Extract the dataframe from the result
df = result[dataset_name]

# Print the dataframe
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings,employment,installment_rate,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,telephone,foreign_worker,class
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,0.0
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,1.0
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,0.0
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,0.0
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,1.0


In [6]:
df.describe()

,duration,credit_amount,installment_rate,residence_since,age,existing_credits,num_dependents,class
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000,0.300000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086,0.458487
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000,0.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000,0.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000,0.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000,1.000000


In [32]:
df.isnull().values.any()

False

## Preprocessing

In [7]:
y_label = "class"
features_to_drop = ["purpose", "duration", "installment_rate" ]

In [8]:
personal_status_to_gender = {
    'A91': 0,
    'A92': 1,
    'A93': 0,
    'A94': 0,
    'A95': 1
}

# Create the gender feature
df['Gender'] = df['personal_status'].map(personal_status_to_gender)

# Drop the personal_status feature
df.drop(columns=['personal_status'], inplace=True)

In [9]:
df = convert_nonzero_to_one(y_label)
df[y_label].value_counts()

class
0.0    700
1.0    300
Name: count, dtype: int64

In [10]:
df = drop_features(features_to_drop)

In [11]:
df.dtypes

checking_status        category
credit_history         category
credit_amount             int32
savings                category
employment             category
other_parties          category
residence_since           int32
property_magnitude     category
age                       int32
other_payment_plans    category
housing                category
existing_credits          int32
job                    category
num_dependents            int32
telephone              category
foreign_worker         category
class                   float64
Gender                    int64
dtype: object

In [12]:
positive_outcome = 1  # Assuming 1 represents the positive outcome

# Calculate counts
total_counts = df['Gender'].value_counts()
positive_counts = df[df[y_label] == 1]['Gender'].value_counts()

# Calculate base rates
base_rates = positive_counts / total_counts

# Display results
for gender, rate in base_rates.items():
    gender_label = 'Female' if gender == 1 else 'Male'
    print(f"Gender: {gender_label}")
    print(f"  Positive outcome ({y_label} = {positive_outcome}) rate: {rate:.2%}")
    print(f"  Total count: {total_counts[gender]}")
    print(f"  Positive count: {positive_counts[gender]}")
    print()

Gender: Male
  Positive outcome (class = 1) rate: 27.68%
  Total count: 690
  Positive count: 191

Gender: Female
  Positive outcome (class = 1) rate: 35.16%
  Total count: 310
  Positive count: 109



## Save as a CSV

In [38]:
df.to_csv(dataset_name + '.csv')